# run dirs ws-out

In [1]:
! for i in `cat ../Orchestration/junk` ; do echo $i; aws s3 ls ws-out/CONUS/$i ;echo '---------------' ; done


Run02_17_2022/
                           PRE r50t7/
---------------
Run02_24_2022/
                           PRE conus_r50t6/
---------------
Run02_25_2022/
                           PRE conus_r50t5/
                           PRE conus_r50t8/
---------------
Run03_01_2022/
                           PRE conus_r37t0/
---------------
Run03_03_2022/
                           PRE conus_r37t1/
                           PRE conus_r37t2/
---------------
Run03_07_2022/
                           PRE conus_r37t3/
---------------
Run03_11_2022/
                           PRE conus_r50t9/
---------------


# output dirs 

In [2]:
! aws s3 ls ws-enduser/CONUS/

                           PRE r37.0_tile0/
                           PRE r37.0_tile1/
                           PRE r37.0_tile2/
                           PRE r37.0_tile3/
                           PRE r50.0_tile5/
                           PRE r50.0_tile6/
                           PRE r50.0_tile7/
                           PRE r50.0_tile8/
                           PRE r50.0_tile9/


In [3]:
import boto3

In [4]:
def s3_bucket_analyze(bucket, prefix):

    objs = []

    print("bucket", bucket)
    print("prefix", prefix)

    bucket_name = bucket
    prefix = prefix

    s3 = boto3.resource('s3')
    bucket = s3.Bucket(bucket_name)
    cnt=0
    storage_class_h = {'STANDARD' : 0,
                   'GLACIER' : 0,
                   'INTELLIGENT_TIERING' : 0,
                  }
    sum_class_h = {'STANDARD' : 0,
                   'GLACIER' : 0,
                   'INTELLIGENT_TIERING' : 0,
                  }
    sum = 0
    for obj in bucket.objects.filter(Prefix=prefix):
        storage_class_h[obj.storage_class] = storage_class_h[obj.storage_class] + 1
        cnt = cnt + 1
        if not cnt%1000:
            print(bucket, "bucket object count = ", cnt, flush=True)
        sum = sum + obj.size
        sum_class_h[obj.storage_class] = sum_class_h[obj.storage_class] + obj.size
        my_obj = {
                'bucket_name':obj.bucket_name,
                'key':obj.key,
                'size':obj.size,
                'class':obj.storage_class
                }
        objs.append(my_obj)

    print ("COUNT=", cnt)
    for ky in storage_class_h.keys():
        print(ky, storage_class_h[ky])
        sum = sum_class_h[ky]
        print(ky, sum_class_h[ky])
        gig = sum/(1024*1024*1024)
        print (ky, "GBYTES=", gig)
        if ky == 'GLACIER': 
            cost=.007
        else: 
            cost=.023
        print (ky, "Cost/Month=", gig * cost)
        print ("----" * 25)
    print("END LOOP")
    my_key = 'STANDARD'
    ret_gbytes = sum_class_h[my_key]/(1024*1024*1024)
    if (ret_gbytes < 1):
        ret_gbytes = 1
    ret_costs = .023 * ret_gbytes
    print('G:', ret_gbytes, ret_costs)
    return objs

In [5]:
bucket='ws-enduser'
my_prefix='CONUS/'

In [6]:
objs = s3_bucket_analyze(bucket, my_prefix)


bucket ws-enduser
prefix CONUS/
s3.Bucket(name='ws-enduser') bucket object count =  1000
s3.Bucket(name='ws-enduser') bucket object count =  2000
s3.Bucket(name='ws-enduser') bucket object count =  3000
s3.Bucket(name='ws-enduser') bucket object count =  4000
s3.Bucket(name='ws-enduser') bucket object count =  5000
s3.Bucket(name='ws-enduser') bucket object count =  6000
s3.Bucket(name='ws-enduser') bucket object count =  7000
s3.Bucket(name='ws-enduser') bucket object count =  8000
s3.Bucket(name='ws-enduser') bucket object count =  9000
s3.Bucket(name='ws-enduser') bucket object count =  10000
s3.Bucket(name='ws-enduser') bucket object count =  11000
s3.Bucket(name='ws-enduser') bucket object count =  12000
s3.Bucket(name='ws-enduser') bucket object count =  13000
s3.Bucket(name='ws-enduser') bucket object count =  14000
s3.Bucket(name='ws-enduser') bucket object count =  15000
s3.Bucket(name='ws-enduser') bucket object count =  16000
s3.Bucket(name='ws-enduser') bucket object count 

s3.Bucket(name='ws-enduser') bucket object count =  142000
s3.Bucket(name='ws-enduser') bucket object count =  143000
s3.Bucket(name='ws-enduser') bucket object count =  144000
s3.Bucket(name='ws-enduser') bucket object count =  145000
s3.Bucket(name='ws-enduser') bucket object count =  146000
s3.Bucket(name='ws-enduser') bucket object count =  147000
s3.Bucket(name='ws-enduser') bucket object count =  148000
s3.Bucket(name='ws-enduser') bucket object count =  149000
s3.Bucket(name='ws-enduser') bucket object count =  150000
s3.Bucket(name='ws-enduser') bucket object count =  151000
s3.Bucket(name='ws-enduser') bucket object count =  152000
s3.Bucket(name='ws-enduser') bucket object count =  153000
s3.Bucket(name='ws-enduser') bucket object count =  154000
s3.Bucket(name='ws-enduser') bucket object count =  155000
s3.Bucket(name='ws-enduser') bucket object count =  156000
s3.Bucket(name='ws-enduser') bucket object count =  157000
s3.Bucket(name='ws-enduser') bucket object count =  1580

In [7]:
objs[0]

{'bucket_name': 'ws-enduser',
 'key': 'CONUS/r37.0_tile0/2000/dd_2000001.tif',
 'size': 826960,
 'class': 'STANDARD'}

In [8]:
new_objs=[]
for o in objs:
    a = o['key'].split('/')
    project = a[0]
    tile = a[1]
    year = a[2]
    basenm = a[3]
    new_o = {
        'bucket_name': o['bucket_name'],
        'project': project,
        'tile': tile,
        'year': year,
        'basename': basenm,
        'size': o['size']
    }
    new_objs.append(new_o)

In [9]:
import pandas as pd

df = pd.DataFrame(new_objs)

In [10]:
df

,bucket_name,project,tile,year,basename,size
0,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000001.tif,826960
1,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000002.tif,827064
2,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000003.tif,826994
3,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000004.tif,830740
4,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000005.tif,827651
...,...,...,...,...,...,...
209505,ws-enduser,CONUS,r50.0_tile9,2020,srf_202008.tif,1295984
209506,ws-enduser,CONUS,r50.0_tile9,2020,srf_202009.tif,1108483
209507,ws-enduser,CONUS,r50.0_tile9,2020,srf_202010.tif,10106845
209508,ws-enduser,CONUS,r50.0_tile9,2020,srf_202011.tif,16757676


In [11]:
tiles = df['tile'].unique()

In [12]:
tiles

array(['r37.0_tile0', 'r37.0_tile1', 'r37.0_tile2', 'r37.0_tile3',
       'r50.0_tile5', 'r50.0_tile6', 'r50.0_tile7', 'r50.0_tile8',
       'r50.0_tile9'], dtype=object)

In [13]:
tdf = df[df['tile'] == tiles[0]]

In [14]:
tdf

,bucket_name,project,tile,year,basename,size
0,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000001.tif,826960
1,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000002.tif,827064
2,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000003.tif,826994
3,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000004.tif,830740
4,ws-enduser,CONUS,r37.0_tile0,2000,dd_2000005.tif,827651
...,...,...,...,...,...,...
34551,ws-enduser,CONUS,r37.0_tile0,2020,srf_2020086.tif,3136211
34552,ws-enduser,CONUS,r37.0_tile0,2020,srf_2020087.tif,1934068
34553,ws-enduser,CONUS,r37.0_tile0,2020,srf_2020088.tif,1220876
34554,ws-enduser,CONUS,r37.0_tile0,2020,srf_2020089.tif,1707382


In [15]:
tdf.describe()

,size
count,3.455600e+04
mean,5.254160e+07
std,3.142912e+07
min,8.269600e+05
25%,1.007028e+07
50%,7.389235e+07
75%,7.449187e+07
max,7.681894e+07


In [16]:
variables = ['etasw','srf', 'dd','netet', 'etc']

In [24]:
import re
def validate_monthly_data(tdf):
    tile = tdf['tile'][0]
    for variable in variables:
        for year in range(2000,2020+1):
            ytdf = tdf[tdf['year'] == str(year)]
            basenames = ytdf['basename'].to_list()
            #print(basenames)
            #print(ytdf)
            #for m in range(1,12+1):
            rematch=f'{variable}_{year}\d\d.tif'
            r = re.compile(rematch)
            my_monthly_list_by_var = [x for x in basenames if r.match(x)]
            print(tile, variable, year, len(my_monthly_list_by_var))

            

In [25]:
validate_monthly_data(tdf)

r37.0_tile0 etasw 2000 12
r37.0_tile0 etasw 2001 12
r37.0_tile0 etasw 2002 12
r37.0_tile0 etasw 2003 12
r37.0_tile0 etasw 2004 12
r37.0_tile0 etasw 2005 12
r37.0_tile0 etasw 2006 12
r37.0_tile0 etasw 2007 12
r37.0_tile0 etasw 2008 12
r37.0_tile0 etasw 2009 12
r37.0_tile0 etasw 2010 12
r37.0_tile0 etasw 2011 12
r37.0_tile0 etasw 2012 12
r37.0_tile0 etasw 2013 12
r37.0_tile0 etasw 2014 12
r37.0_tile0 etasw 2015 12
r37.0_tile0 etasw 2016 12
r37.0_tile0 etasw 2017 12
r37.0_tile0 etasw 2018 12
r37.0_tile0 etasw 2019 12
r37.0_tile0 etasw 2020 12
r37.0_tile0 srf 2000 12
r37.0_tile0 srf 2001 12
r37.0_tile0 srf 2002 12
r37.0_tile0 srf 2003 12
r37.0_tile0 srf 2004 12
r37.0_tile0 srf 2005 12
r37.0_tile0 srf 2006 12
r37.0_tile0 srf 2007 12
r37.0_tile0 srf 2008 12
r37.0_tile0 srf 2009 12
r37.0_tile0 srf 2010 12
r37.0_tile0 srf 2011 12
r37.0_tile0 srf 2012 12
r37.0_tile0 srf 2013 12
r37.0_tile0 srf 2014 12
r37.0_tile0 srf 2015 12
r37.0_tile0 srf 2016 12
r37.0_tile0 srf 2017 12
r37.0_tile0 srf 2018 8